In [1]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:

!pip install -q transformers datasets


In [3]:
# Caminho base do Drive
base_path = "/content/drive/MyDrive/AKCIT - NLP (Desidentificação e identificação de dados sensíveis)/modelos-selecionados"

In [ ]:
import os
import json
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# Caminho base do Drive
base_path = "/content/drive/MyDrive/AKCIT - NLP (Desidentificação e identificação de dados sensíveis)/modelos-selecionados"

# === Carregar modelo BERTimbau salvo localmente ===
bert_model_dir = os.path.join(base_path, "model_roberta")  # deve conter config.json, tokenizer_config.json, etc.
tokenizer = AutoTokenizer.from_pretrained(bert_model_dir)
model_bert = AutoModelForTokenClassification.from_pretrained(bert_model_dir)

# Criar pipeline de inferência
bert_pipeline = pipeline(
    "ner",
    model=model_bert,
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

Device set to use cpu


In [ ]:
import os
import re
import sys
import glob
import pandas as pd
from collections import defaultdict

# ==== Paths/config ====
in_csv   = base_path + '/top_10_consolidado_maraba_coluna_interesse_clean.csv'
out_dir  = base_path + '/masked_parts'   # pasta onde salvaremos as partes
prefix   = 'top10_masked_part'           # prefixo do nome do arquivo
chunk_sz = 5000                          # linhas por parte

os.makedirs(out_dir, exist_ok=True)

# ==== Função de máscara (sem logs verbosos) ====
def mask_entities_in_text(text: str):
    if not isinstance(text, str) or not text.strip():
        return text
    try:
        ents = bert_pipeline(text) or []
    except Exception:
        return text

    counters = defaultdict(int)

    if ents and all(('start' in e and 'end' in e and isinstance(e['start'], int) and isinstance(e['end'], int)) for e in ents):
        ents_sorted = sorted(ents, key=lambda e: e['start'], reverse=True)
        masked = text
        for e in ents_sorted:
            group = str(e.get('entity_group', 'ENT')).upper()
            counters[group] += 1
            placeholder = f"[{group} {counters[group]:02d}]"
            s, ed = e['start'], e['end']
            if 0 <= s < ed <= len(masked):
                masked = masked[:s] + placeholder + masked[ed:]
        return masked

    masked = text
    for e in ents:
        group = str(e.get('entity_group', 'ENT')).upper()
        word = str(e.get('word', '')).strip()
        if not word:
            continue
        clean = re.sub(r'\s+', ' ', word.replace('##', ''))
        if len(clean) < 2:
            continue
        counters[group] += 1
        placeholder = f"[{group} {counters[group]:02d}]"
        pattern = r'\b' + re.escape(clean) + r'\b'
        masked = re.sub(pattern, placeholder, masked, flags=re.IGNORECASE)
    return masked

# ==== Leitura do dataset (inteiro; se for muito grande, podemos adaptar para ler em streaming) ====
df = pd.read_csv(in_csv)

# ==== Descobrir de onde retomar ====
# Procuramos arquivos existentes no padrão: top10_masked_part_00001.csv, etc.
def list_parts():
    files = sorted(glob.glob(os.path.join(out_dir, f"{prefix}_*.csv")))
    # Mantém apenas os que seguem o padrão _NNNNN.csv
    return [f for f in files if re.search(rf"{re.escape(prefix)}_(\d+)\.csv$", os.path.basename(f))]

existing = list_parts()

def part_number_from_name(fname: str) -> int:
    m = re.search(rf"{re.escape(prefix)}_(\d+)\.csv$", os.path.basename(fname))
    return int(m.group(1)) if m else 0

if existing:
    last_file = existing[-1]
    last_part = part_number_from_name(last_file)
    try:
        last_df = pd.read_csv(last_file)
        rows_in_last = len(last_df)
    except Exception:
        # Se não conseguir ler, consideramos 0 para evitar duplicação
        rows_in_last = 0
else:
    last_file = None
    last_part = 0
    rows_in_last = 0

# Quantas linhas globais já foram processadas (completas + a parte incompleta)
processed_global = (last_part - 1) * chunk_sz + rows_in_last if last_part > 0 else 0
total = len(df)

print(f"🔁 Retomando a partir da linha {processed_global}/{total} "
      f"(parte atual: {last_part if last_part>0 else 1}, preenchidas: {rows_in_last}/{chunk_sz})")

# ==== Loop de processamento com gravação por partes ====
# Estratégia:
# - Se há uma parte existente e incompleta (rows_in_last < chunk_sz), vamos COMPLETAR essa parte (append).
# - Depois abrimos novas partes sequenciais até cobrir todo o dataset.

current_part = last_part if last_part > 0 else 1
rows_filled_in_part = rows_in_last
i = processed_global

# Função para gravar com atomicidade
def atomic_write(df_to_write: pd.DataFrame, dest_path: str, mode: str, header: bool):
    tmp_path = dest_path + ".tmp"
    df_to_write.to_csv(tmp_path, index=False, encoding='utf-8-sig', mode='w' if header else 'a', header=header)
    # Se modo append, garantimos que arquivo-base exista (criado no primeiro write header=True) — por isso header controla.
    # Renomeia: se header=True, simplesmente move tmp para destino; se header=False, anexamos e depois removemos tmp.
    if header:
        # Criação/reescrita do arquivo (para parte nova)
        os.replace(tmp_path, dest_path)
    else:
        # Append: vamos abrir o existente e anexar o tmp de forma eficiente:
        # Aqui simplificamos: lemos o tmp e anexamos com open() .write(); para evitar overhead,
        # regravamos em modo append direto via pandas já acima (mode='a'), então só precisamos mover sobre o próprio arquivo.
        # Como pandas já gravou no tmp, precisamos concatenar?
        # Melhor abordagem: escrever direto no destino em modo append sem atomicidade por linha.
        # Para atomicidade simples, faremos: escrever direto (abaixo alternativa mais simples e robusta):
        pass

# Para simplificar: faremos duas rotas:
# - Para parte nova (header=True): escrevemos atomicamente (tmp -> replace).
# - Para append em parte existente: escreveremos diretamente com df.to_csv(..., mode='a', header=False).

def write_append(dest_path: str, df_to_write: pd.DataFrame):
    df_to_write.to_csv(dest_path, index=False, encoding='utf-8-sig', mode='a', header=False)

# Barra de progresso leve (sem depender de tqdm)
def progress(i_now: int):
    if i_now % 1000 == 0 or i_now == total:
        pct = (i_now / total * 100) if total else 100
        print(f"… {i_now}/{total} ({pct:.1f}%)")

while i < total:
    # Se a parte atual ainda está incompleta, calculamos quanto falta para completá-la
    room = chunk_sz - rows_filled_in_part
    take = min(room, total - i)
    if take <= 0:
        # inicia nova parte
        current_part += 1
        rows_filled_in_part = 0
        continue

    # fatia a próxima janela
    batch_df = df.iloc[i:i+take].copy()

    # aplica mask apenas à coluna 'relato' e escreve a coluna final 'relato_masked'
    masked = [mask_entities_in_text(x) for x in batch_df['relato']]
    batch_df['relato_masked'] = masked

    # caminho da parte atual
    part_name = os.path.join(out_dir, f"{prefix}_{current_part:05d}.csv")

    if rows_filled_in_part == 0 and (last_part == 0 or current_part > last_part):
        # Parte NOVA → escrever com header (atomicamente)
        tmp_path = part_name + ".tmp"
        batch_df.to_csv(tmp_path, index=False, encoding='utf-8-sig')
        os.replace(tmp_path, part_name)
    else:
        # Parte EXISTENTE e incompleta → append sem header
        write_append(part_name, batch_df)

    i += take
    rows_filled_in_part += take
    progress(i)

    # Se completamos a parte, iniciamos a próxima
    if rows_filled_in_part >= chunk_sz:
        current_part += 1
        rows_filled_in_part = 0

print("✅ Finalizado.")
print(f"📁 Partes gravadas em: {out_dir}")


🔁 Retomando a partir da linha 40000/52629 (parte atual: 8, preenchidas: 5000/5000)
… 45000/52629 (85.5%)
… 50000/52629 (95.0%)
… 52629/52629 (100.0%)
✅ Finalizado.
📁 Partes gravadas em: /content/drive/MyDrive/AKCIT - NLP (Desidentificação e identificação de dados sensíveis)/modelos-selecionados/masked_parts


In [ ]:
import os
import re
import glob
import sys
import pandas as pd

# ==== Config ====
out_dir  = base_path + '/masked_parts'
prefix   = 'top10_masked_part'
merged_csv    = base_path + '/top10_masked_merged.csv'
merged_parquet = base_path + '/top10_masked_merged.parquet'  # opcional

os.makedirs(out_dir, exist_ok=True)

def list_parts():
    files = sorted(glob.glob(os.path.join(out_dir, f"{prefix}_*.csv")))
    return [f for f in files if re.search(rf"{re.escape(prefix)}_(\d+)\.csv$", os.path.basename(f))]

def part_number_from_name(fname: str) -> int:
    m = re.search(rf"{re.escape(prefix)}_(\d+)\.csv$", os.path.basename(fname))
    return int(m.group(1)) if m else 0

parts = list_parts()
if not parts:
    raise SystemExit(f"⚠️ Nenhum arquivo encontrado em {out_dir} com prefixo {prefix}_*.csv")

# ordena por número da parte (garante ordem cronológica)
parts = sorted(parts, key=part_number_from_name)

print(f"🔗 Unindo {len(parts)} partes de: {out_dir}")
print(f"   Exemplo: {os.path.basename(parts[0])} ... {os.path.basename(parts[-1])}")

# ====== Passo 1: detectar esquema "superset" (todas as colunas que aparecem em alguma parte) ======
all_cols = set()
valid_parts = []
for p in parts:
    try:
        head = pd.read_csv(p, nrows=0)
        all_cols.update(list(head.columns))
        valid_parts.append(p)
    except Exception as e:
        print(f"⚠️ Ignorando parte corrompida ou ilegível: {p} ({e})")

if not valid_parts:
    raise SystemExit("❌ Nenhuma parte válida para unir.")

all_cols = list(all_cols)  # ordem arbitrária; depois tentamos ordenar melhor
# Tenta priorizar algumas colunas comuns
prefer_first = ['relato', 'relato_masked', 'consolidado', 'municipios', 'distrito', 'regionais', 'bairros']
ordered = [c for c in prefer_first if c in all_cols] + [c for c in all_cols if c not in prefer_first]
all_cols = ordered

# ====== Passo 2: leitura incremental, concat e deduplicação ======
frames = []
rows_total = 0

for i, p in enumerate(valid_parts, start=1):
    try:
        dfp = pd.read_csv(p)
    except Exception as e:
        print(f"⚠️ Falha ao ler {p}, ignorando. Erro: {e}")
        continue

    # Garante todas as colunas do superset; colunas ausentes entram como NaN
    missing = [c for c in all_cols if c not in dfp.columns]
    if missing:
        for c in missing:
            dfp[c] = pd.NA
        # Reordena colunas
        dfp = dfp[all_cols]
    else:
        # Apenas reordena
        dfp = dfp[all_cols]

    frames.append(dfp)
    rows_total += len(dfp)

    # progresso leve
    if i % 5 == 0 or i == len(valid_parts):
        pct = (i / len(valid_parts)) * 100
        print(f"… lidas {i}/{len(valid_parts)} partes ({pct:.1f}%), linhas acumuladas: {rows_total}")

if not frames:
    raise SystemExit("❌ Nenhum dado carregado das partes.")

merged = pd.concat(frames, ignore_index=True)

# ====== Deduplicação ======
# Preferimos deduplicar por 'relato_masked' se existir; senão caímos para 'relato'.
if 'relato_masked' in merged.columns:
    key_subset = ['relato_masked']
elif 'relato' in merged.columns:
    key_subset = ['relato']
else:
    # Sem colunas de texto conhecidas — dedup por todas as colunas (mais custoso)
    key_subset = None

before = len(merged)
if key_subset:
    merged.drop_duplicates(subset=key_subset, keep='first', inplace=True)
else:
    merged.drop_duplicates(keep='first', inplace=True)
after = len(merged)
removed = before - after

print(f"🧹 Removidas {removed} duplicações; total final: {after} linhas.")

# ====== Salvar ======
# CSV
tmp_csv = merged_csv + ".tmp"
merged.to_csv(tmp_csv, index=False, encoding='utf-8-sig')
os.replace(tmp_csv, merged_csv)
print(f"✅ CSV unido salvo em: {merged_csv}")

# Parquet (opcional; comente se não quiser)
try:
    merged.to_parquet(merged_parquet, index=False)
    print(f"✅ Parquet salvo em: {merged_parquet}")
except Exception as e:
    print(f"ℹ️ Não foi possível salvar Parquet (ok ignorar): {e}")


🔗 Unindo 11 partes de: /content/drive/MyDrive/AKCIT - NLP (Desidentificação e identificação de dados sensíveis)/modelos-selecionados/masked_parts
   Exemplo: top10_masked_part_00001.csv ... top10_masked_part_00011.csv
… lidas 5/11 partes (45.5%), linhas acumuladas: 25000
… lidas 10/11 partes (90.9%), linhas acumuladas: 50000
… lidas 11/11 partes (100.0%), linhas acumuladas: 52629
🧹 Removidas 24 duplicações; total final: 52605 linhas.
✅ CSV unido salvo em: /content/drive/MyDrive/AKCIT - NLP (Desidentificação e identificação de dados sensíveis)/modelos-selecionados/top10_masked_merged.csv
✅ Parquet salvo em: /content/drive/MyDrive/AKCIT - NLP (Desidentificação e identificação de dados sensíveis)/modelos-selecionados/top10_masked_merged.parquet


In [ ]:
import pandas as pd

df = pd.read_csv(base_path+'/top10_masked_merged.csv')


df['label'] = df['consolidado'].str.strip()
df['text'] = df['relato']

dft = df[['text','label']]
dft.head()


import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

### convert to Huggingface dataset
#dataset = Dataset(pa.Table.from_pandas(selected_rows))#
#todas linhas dessa
dataset = Dataset(pa.Table.from_pandas(dft))
dataset = dataset.class_encode_column("label")

dataset.save_to_disk(f"{base_path}/labelTextMarabaRank10.hf")

print(dataset)

Casting to class labels:   0%|          | 0/52605 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/52605 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 52605
})


In [ ]:
import pandas as pd

df = pd.read_csv(base_path+'/top10_masked_merged.csv')


df['label'] = df['consolidado'].str.strip()
df['text'] = df['relato_masked']

dft = df[['text','label']]
dft.head()


import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

### convert to Huggingface dataset
#dataset = Dataset(pa.Table.from_pandas(selected_rows))#
#todas linhas dessa
dataset = Dataset(pa.Table.from_pandas(dft))
dataset = dataset.class_encode_column("label")

dataset.save_to_disk(f"{base_path}/labelTextMarabaRank10masked.hf")

print(dataset)

Casting to class labels:   0%|          | 0/52605 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/52605 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 52605
})


##Divisão do dataset

In [ ]:
!pip install datasets
from datasets import load_from_disk
from datasets import DatasetDict
import datetime
import os

In [ ]:
name = "labelTextMarabaRank10masked"
name_dataset = f"{base_path}/classification_dataset/{name}.hf"
data = load_from_disk(name_dataset)

In [ ]:
seed = 42
column_to_stratify = "label"

train_testvalid = data.train_test_split(train_size=0.8, seed=seed, stratify_by_column=column_to_stratify)

# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(train_size=0.5, seed=seed, stratify_by_column=column_to_stratify)

# gather everyone if you want to have a single DatasetDict
#bert_dataset = concatenate_datasets([bookcorpus, wiki])
data2 = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'unsupervised': test_valid['train']})

In [ ]:
data2.save_to_disk(f"{base_path}/classification_dataset/{name}_split.hf")

Saving the dataset (0/1 shards):   0%|          | 0/42084 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5261 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5260 [00:00<?, ? examples/s]

In [15]:
from datasets import load_from_disk
from datasets import DatasetDict
import datetime
import os

name = "labelTextMarabaRank10masked_split"
name_dataset = f"{base_path}/classification_dataset/{name}.hf"
data = load_from_disk(name_dataset)

In [17]:
data['train'][:5]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 42084
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5261
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 5260
    })
})